In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
with open('/Users/amnam/.secret/yelp_api.json') as f:
    login = json.load(f)

yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)


In [3]:
# set our API call parameters 
LOCATION = 'Dearborn,MI'
TERM = 'Fried Chicken'

In [4]:
JSON_FILE = "Data/results_in_progress_MI_firedchicken.json"
JSON_FILE

'Data/results_in_progress_MI_firedchicken.json'

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_MI_firedchicken.json not found. Saving empty list to file.


In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
## How many results total?
total_results = results['total']
total_results

564

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [10]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

29

In [11]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [12]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/29 [00:00<?, ?it/s]

In [13]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,Cu3KC6x3WUFTzcz32TkbfA,ex-wifes-famous-chicken-dearborn-heights,Ex-Wife's Famous Chicken,https://s3-media4.fl.yelpcdn.com/bphoto/zuWl_g...,False,https://www.yelp.com/biz/ex-wifes-famous-chick...,48,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",4.0,"{'latitude': 42.32736, 'longitude': -83.25863}",[],$$,"{'address1': '22444 Ford Rd', 'address2': None...",+13138801180,(313) 880-1180,4003.498796
1,7orZgupOKK9I7fYekdhIhA,daves-hot-chicken-dearborn,Dave's Hot Chicken,https://s3-media2.fl.yelpcdn.com/bphoto/QAKdyn...,False,https://www.yelp.com/biz/daves-hot-chicken-dea...,89,"[{'alias': 'halal', 'title': 'Halal'}, {'alias...",4.0,"{'latitude': 42.30547, 'longitude': -83.2485}","[delivery, pickup]",$$,"{'address1': '22208 Michigan Ave', 'address2':...",+13133800699,(313) 380-0699,3046.090322
2,CYj6Xynv7CwUZPSYoUzCVw,mr-chicken-dearborn-heights,Mr Chicken,https://s3-media3.fl.yelpcdn.com/bphoto/tq_4J1...,False,https://www.yelp.com/biz/mr-chicken-dearborn-h...,92,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 42.33103, 'longitude': -83.27218}",[delivery],$$,"{'address1': '6000 N Telegraph Rd', 'address2'...",+13132770100,(313) 277-0100,5204.792531
3,H-Ek50bSn5KCBQ1_tVmQAA,gus-world-famous-fried-chicken-westland,Gus' World Famous Fried Chicken,https://s3-media2.fl.yelpcdn.com/bphoto/Ol6V_O...,False,https://www.yelp.com/biz/gus-world-famous-frie...,296,"[{'alias': 'southern', 'title': 'Southern'}, {...",4.0,"{'latitude': 42.32409, 'longitude': -83.39072}","[delivery, pickup]",$$,"{'address1': '35505 Ford Rd', 'address2': '', ...",+17347286170,(734) 728-6170,14632.818763
4,ejoVEHzo2b0VNpa90uK44g,fat-daddys-hot-chicken-and-waffles-riverview,Fat Daddy's Hot Chicken & Waffles,https://s3-media1.fl.yelpcdn.com/bphoto/u_LlbE...,False,https://www.yelp.com/biz/fat-daddys-hot-chicke...,139,"[{'alias': 'waffles', 'title': 'Waffles'}, {'a...",4.5,"{'latitude': 42.174526, 'longitude': -83.188452}","[delivery, pickup]",NaN,"{'address1': '18283 Fort St', 'address2': '', ...",+17342883051,(734) 288-3051,15678.060298


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
15,s8x1eRJyIKX0xrjNLTFG7w,the-himalayan-flames-dearborn,The Himalayan Flames,https://s3-media2.fl.yelpcdn.com/bphoto/QwcaMl...,False,https://www.yelp.com/biz/the-himalayan-flames-...,220,"[{'alias': 'himalayan', 'title': 'Himalayan/Ne...",4.5,"{'latitude': 42.305171, 'longitude': -83.249325}","[delivery, pickup]",$$,"{'address1': '22266 Michigan Ave', 'address2':...",+13139081193,(313) 908-1193,3140.963700
16,ozZWtgcx1XbnGz4uVm9vmg,captain-jays-fish-and-chicken-detroit-2,Captain Jay's Fish and Chicken,https://s3-media3.fl.yelpcdn.com/bphoto/aI2pUj...,False,https://www.yelp.com/biz/captain-jays-fish-and...,11,"[{'alias': 'newamerican', 'title': 'American (...",2.5,"{'latitude': 42.3435941629798, 'longitude': -8...",[delivery],NaN,"{'address1': '18440 W Warren Ave', 'address2':...",+13139820100,(313) 982-0100,3300.722848
17,2zHyxn8w_jIJbjibwIUgoQ,penny-reds-detroit,Penny Red's,https://s3-media1.fl.yelpcdn.com/bphoto/qzgfL9...,False,https://www.yelp.com/biz/penny-reds-detroit?ad...,116,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",3.5,"{'latitude': 42.3348666, 'longitude': -83.0482...","[delivery, pickup]",$$,"{'address1': '1445 Farmer St', 'address2': '',...",,,13748.256793
18,Bzk_iZbNigTTcZdr2F8khw,wishbone-chicken-and-ribs-allen-park,Wishbone Chicken & Ribs,https://s3-media3.fl.yelpcdn.com/bphoto/l7ie09...,False,https://www.yelp.com/biz/wishbone-chicken-and-...,34,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",2.5,"{'latitude': 42.256335, 'longitude': -83.225526}",[],$$,"{'address1': '18617 Ecorse Rd', 'address2': ''...",+13133839600,(313) 383-9600,6529.038806
19,EKL28BewkwaF3webb9mWtg,rafics-falafel-dearborn,Rafic's Falafel,https://s3-media3.fl.yelpcdn.com/bphoto/1vzqpD...,False,https://www.yelp.com/biz/rafics-falafel-dearbo...,99,"[{'alias': 'lebanese', 'title': 'Lebanese'}, {...",3.5,"{'latitude': 42.34133, 'longitude': -83.17692}","[delivery, pickup]",$,"{'address1': '6905 Schaefer Rd', 'address2': '...",+13135515046,(313) 551-5046,4224.314500


In [14]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [15]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_MI_firedchicken.csv.gz', compression='gzip',index=False)